In [4]:
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
from selenium.common.exceptions import TimeoutException
import pandas as pd 
import numpy as np

In [5]:
driver = webdriver.Chrome()

In [6]:
driver.get('https://www.cars24.com/')

In [7]:
wait = WebDriverWait(driver,10)

## Going to location Page

In [8]:
try:
    loc_btn = wait.until(EC.presence_of_element_located(('class name', '_1K8Qe')))
    loc_btn.click()
except TimeoutException:
    set_loc_btn = wait.until(EC.presence_of_element_located(('class name', '_2SBi0')))
    set_loc_btn.click()

    loc_btn = wait.until(EC.presence_of_element_located(('class name', '_1K8Qe')))
    loc_btn.click()

## Choosing the location

In [6]:
loc_cfm = wait.until(EC.presence_of_element_located(('class name','_1oBNU')))
loc_cfm.click()

## Selecting buy old cars

In [7]:
buy_car_btn = wait.until(EC.presence_of_element_located(('class name','u31Ay')))
buy_car_btn.click()

## Selecting Bangalore

In [8]:
bangalore_btn = wait.until(EC.presence_of_element_located(('xpath','//a[@href="https://www.cars24.com/buy-used-cars-bangalore/"]')))
bangalore_btn.click()

In [9]:
driver.maximize_window()

In [10]:
switch = driver.window_handles[0]
driver.switch_to.window(switch)

## Scrolling to All brands

In [11]:
all_brands_ele = wait.until(EC.presence_of_all_elements_located(('class name','_1XyjH')))[-1]
driver.execute_script("arguments[0].scrollIntoView(true);", all_brands_ele)

## Clicking the brands we require

In [9]:
def brand_selector(name):
    try:
        brands_ele = wait.until(EC.presence_of_all_elements_located(('class name', 'rTjwY')))
        checkbox_brands_ele = wait.until(EC.presence_of_all_elements_located(('class name', '_2dra0')))
        
        diff = len(checkbox_brands_ele) - len(brands_ele)
        
        for index, brand in enumerate(brands_ele):
            if brand.text.split('\n')[0].strip().lower() == name.strip().lower():
                checkbox_brands_ele[diff + index].click()
                return True
        
        print("Sorry, we are unable to find your desired brand.")
        return False
    
    except Exception as e:
        print(f"An error occurred: {e}")
        return False


In [13]:
any_car_selected = False

## Taking input of car name(Hyundai)

In [14]:
car_name = "hyundai"
found = brand_selector(car_name)
if found:
    any_car_selected = True

## Extracting Details of Hyundai

In [15]:
def total_models():
    total_model = int(wait.until(EC.presence_of_element_located(('class name','_1Bq4a'))).text.split()[0])
    return total_model

total = total_models()
print(f"The total number of models of {car_name} is {total}")

The total number of models of hyundai is 319


In [16]:
def data_extraction(total_models):
    
    
    for i in range(0,100000,2000):
        driver.execute_script(f"window.scrollTo(0,{i})")
        
        year_models = wait.until(EC.presence_of_all_elements_located(('class name','_11dVb')))
        driven_fuel_types = wait.until(EC.presence_of_all_elements_located(('class name','_3J2G-')))
        prices = wait.until(EC.presence_of_all_elements_located(('class name','_3RL-I')))
        
        if len(year_models)==total_models and len(driven_fuel_types)==total_models and len(prices)==total_models:
            print("Successfully colllected all data")
    
            driver.execute_script("window.scrollTo(0,0)")

            return [year_models,driven_fuel_types,prices]
            
    return []

In [20]:
datas = data_extraction(total)

Successfully colllected all data


In [21]:
def gathering_data(arr1, arr2, arr3):
    years = []
    models = []
    distance_drivens = []
    fuels = []
    transmissions = []
    prices = []

    print("Processing arr1:")
    for data in arr1:
        raw_text = data.text
        print(f"Raw text: '{raw_text}'")  # Debugging line

        datas = raw_text.split()
        if len(datas) >= 2:
            year, model = datas[0], " ".join(datas[1:])
            years.append(year)
            models.append(model)
        else:
            print(f"Unexpected format in arr1: {datas}")  # Debugging line

    print("Processing arr2:")
    for data in arr2:
        raw_text = data.text
        print(f"Raw text: '{raw_text}'")  # Debugging line

        datas = raw_text.split('\n')
        if len(datas) >= 3:
            dist, fuel, typ = datas[0], datas[1], datas[2]
            distance_drivens.append(dist)
            fuels.append(fuel)
            transmissions.append(typ)
        else:
            print(f"Unexpected format in arr2: {datas}")  # Debugging line

    print("Processing arr3:")
    for data in arr3:
        raw_text = data.text
        print(f"Raw text: '{raw_text}'")  # Debugging line
        prices.append(raw_text)

    return years, models, distance_drivens, fuels, transmissions, prices


In [22]:
years1, models1 , distance_drivens1 , fuels1 , transmissions1 , prices1 =  gathering_data(datas[0], datas[1], datas[2])

Processing arr1:
Raw text: '2017 Hyundai Creta SX PLUS AT 1.6 PETROL'
Raw text: '2017 Hyundai Eon ERA PLUS (O)'
Raw text: '2019 Hyundai Creta SX 1.6 DIESEL'
Raw text: '2019 Hyundai NEW SANTRO MAGNA'
Raw text: '2021 Hyundai NEW I20 ASTA 1.0 TURBO IMT DUAL TONE'
Raw text: '2019 Hyundai Creta SX AT 1.6 PETROL'
Raw text: '2016 Hyundai Creta SX PLUS AT 1.6 PETROL'
Raw text: '2012 Hyundai i20 SPORTZ 1.2'
Raw text: '2016 Hyundai i20 Active 1.4 SX'
Raw text: '2021 Hyundai Verna SX (O) 1.5 VTVT IVT'
Raw text: '2018 Hyundai Creta SX PLUS AT 1.6 PETROL'
Raw text: '2018 Hyundai Grand i10 ASTA 1.2 KAPPA VTVT'
Raw text: '2016 Hyundai Grand i10 SPORTZ 1.2 KAPPA VTVT'
Raw text: '2020 Hyundai NEW I20 Asta 1.0 GDI Turbo IMT'
Raw text: '2019 Hyundai Creta SX AT 1.6 PETROL'
Raw text: '2019 Hyundai NEW SANTRO SPORTZ MT'
Raw text: '2015 Hyundai Elite i20 ASTA 1.2'
Raw text: '2022 Hyundai GRAND I10 NIOS SPORTZ 1.2 KAPPA VTVT CNG'
Raw text: '2013 Hyundai Verna FLUIDIC 1.6 CRDI SX'
Raw text: '2015 Hyundai Elit

## Taking input of car name(Honda)

In [23]:
if  any_car_selected:
    found = brand_selector(car_name)
    any_car_selected = False

In [24]:
car_name = "honda"
found = brand_selector(car_name)
if found:
    any_car_selected = True

## Extracting Details of Honda

In [31]:
driver.refresh()
time.sleep(2)

In [32]:
total = total_models()
print(f"The total number of models of {car_name} is {total}")

The total number of models of honda is 114


In [33]:
datas = data_extraction(total)

Successfully colllected all data


In [34]:
years2, models2 , distance_drivens2 , fuels2 , transmissions2 , prices2 =  gathering_data(datas[0],datas[1],datas[2])

Processing arr1:
Raw text: '2018 Honda BR-V 1.5L I-VTEC V'
Raw text: '2019 Honda City 1.5L I-VTEC ZX CVT'
Raw text: '2018 Honda Amaze 1.2L I-VTEC VX'
Raw text: '2020 Honda Jazz 1.2L I-VTEC ZX'
Raw text: '2012 Honda Jazz 1.2L I-VTEC X'
Raw text: '2013 Honda Brio S MT'
Raw text: '2018 Honda Amaze 1.2L I-VTEC S'
Raw text: '2013 Honda Brio S MT'
Raw text: '2015 Honda City 1.5L I-VTEC SV CVT'
Raw text: '2019 Honda Jazz 1.2L I-VTEC V'
Raw text: '2019 Honda Civic 1.8L I-VTEC ZX CVT'
Raw text: '2017 Honda WR-V 1.2L I-VTEC VX MT'
Raw text: '2014 Honda City 1.5L I-VTEC SV'
Raw text: '2019 Honda Civic 1.8L I-VTEC ZX CVT'
Raw text: '2014 Honda Brio VX AT'
Raw text: '2015 Honda City 1.5L I-VTEC VX'
Raw text: '2017 Honda Jazz 1.2L I-VTEC V AT'
Raw text: '2020 Honda Amaze 1.2L I-VTEC VX'
Raw text: '2015 Honda Jazz 1.2L I-VTEC SV'
Raw text: '2012 Honda City 1.5L I-VTEC V MT'
Raw text: '2019 Honda Amaze 1.5L I-DTEC V'
Raw text: '2016 Honda City 1.5L I-VTEC V MT'
Raw text: '2015 Honda Jazz 1.2L I-VTEC V

## Taking input of car name(Tata)

In [35]:
all_brands_ele = wait.until(EC.presence_of_all_elements_located(('class name','_1XyjH')))[-1]
driver.execute_script("arguments[0].scrollIntoView(true);", all_brands_ele)

In [36]:
if  any_car_selected:
    found = brand_selector(car_name)
    any_car_selected = False

In [37]:
car_name = "tata"
found = brand_selector(car_name)
if found:
    any_car_selected = True

## Extracting Details of Tata

In [46]:
driver.refresh()
time.sleep(2)

In [47]:
total = total_models()
print(f"The total number of models of {car_name} is {total}")

The total number of models of tata is 95


In [48]:
datas = data_extraction(total)

Successfully colllected all data


In [49]:
years3, models3 , distance_drivens3 , fuels3 , transmissions3 , prices3 =  gathering_data(datas[0],datas[1],datas[2])

Processing arr1:
Raw text: '2023 Tata PUNCH CREATIVE IRA AMT 1.2 RTN DUAL TONE'
Raw text: '2020 Tata NEXON XZA PLUS (O) PETROL'
Raw text: '2022 Tata NEXON XZA PLUS PETROL'
Raw text: '2016 Tata Tiago XZ PETROL'
Raw text: '2022 Tata NEXON XZ PLUS (PREMIUM) PETROL JET'
Raw text: '2021 Tata ALTROZ XE PETROL'
Raw text: '2023 Tata PUNCH CREATIVE IRA AMT 1.2 RTN DUAL TONE'
Raw text: '2016 Tata Tiago XZ PETROL'
Raw text: '2021 Tata ALTROZ XE PETROL'
Raw text: '2016 Tata Tiago XZ PETROL'
Raw text: '2016 Tata Tiago XZ PETROL'
Raw text: '2018 Tata Tiago XZ PETROL'
Raw text: '2018 Tata TIGOR XZA PETROL'
Raw text: '2022 Tata PUNCH CREATIVE IRA AMT 1.2 RTN DUAL TONE'
Raw text: '2022 Tata Safari XZA PLUS DARK EDITION'
Raw text: '2017 Tata Tiago XT PETROL'
Raw text: '2023 Tata PUNCH CREATIVE iRA PACK MT'
Raw text: '2022 Tata ALTROZ XZA PLUS'
Raw text: '2020 Tata ALTROZ XZ PETROL'
Raw text: '2021 Tata Tiago XZ PLUS DUAL TONE PETROL'
Raw text: '2017 Tata NEXON XZ PLUS PETROL DUAL TONE'
Raw text: '2017 T

## Collecting all data in variables

In [50]:
finals = []

In [51]:
for i,j,k,l,m,n in zip(years1,models1,distance_drivens1,fuels1,transmissions1,prices1):
    finals.append([i,"Hyundai",j,k,l,m,n,"Bangalore"])

In [52]:
for i,j,k,l,m,n in zip(years2,models2,distance_drivens2,fuels2,transmissions2,prices2):
    finals.append([i,"Honda",j,k,l,m,n,"Bangalore"])

In [53]:
for i,j,k,l,m,n in zip(years3,models3,distance_drivens3,fuels3,transmissions3,prices3):
    finals.append([i,"Tata",j,k,l,m,n,"Bangalore"])

In [54]:
import pandas as pd

In [55]:
df = pd.DataFrame(finals,columns=['Year','Car','Model','Kilometre Driven','Fuel Type','Transmission Type','Price','Location'])

In [56]:
df.head(5)

,Year,Car,Model,Kilometre Driven,Fuel Type,Transmission Type,Price,Location
0,2017,Hyundai,Hyundai Creta SX PLUS AT 1.6 PETROL,"98,493 KM",PETROL,AUTOMATIC,₹9.73 Lakh,Bangalore
1,2017,Hyundai,Hyundai Eon ERA PLUS (O),"33,963 KM",PETROL,MANUAL,₹3.81 Lakh,Bangalore
2,2019,Hyundai,Hyundai Creta SX 1.6 DIESEL,"43,987 KM",DIESEL,MANUAL,₹11.52 Lakh,Bangalore
3,2019,Hyundai,Hyundai NEW SANTRO MAGNA,"33,609 KM",PETROL,MANUAL,₹4.39 Lakh,Bangalore
4,2021,Hyundai,Hyundai NEW I20 ASTA 1.0 TURBO IMT DUAL TONE,"26,797 KM",PETROL,MANUAL,₹10.21 Lakh,Bangalore


In [57]:
df.to_csv("Bangalore-Car details.csv",index=False)

In [58]:
driver.quit()

In [59]:
from IPython.display import FileLink

FileLink("Bangalore-Car details.csv")

C:\Users\rudra\Bangalore-Car details.csv